https://youtu.be/sDCRB_kzkpg?si=Y3yQK_nid_fGxgh2

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import matplotlib.pyplot as plt

Bubble detecting algorithm developed by Phillips and Yu (2011) and Phillips and al. (2015). It is based on the augmented Dickey-Fuller test, more precisely on the BSADF (Backward Sub Augmented Dickey Fuller test)(backard rolled subsamples test)

In [2]:
# downloading data
ticker = 'DOGE-USD' # doge coin crypto
start = '2019-09-01'
end = '2021-06-15'
prices = yf.download(ticker, start, end)['Close']

[*********************100%%**********************]  1 of 1 completed


In [3]:
# parameters
r0 = int(len(prices) * 0.1) # smallest subsample for ADF test : 10% of the data set

# lag for the augmented dickey fuller test (ADF)
adf_lags = 3 # short term autocorrelation (cryptos, small caps, gamestop etc...)

# critical value of the right tail of ADF test (95%) from Phillips and al. (2015)
crit = 1.49 # if the result of the test exceed this value, with 95% certainty, there is a bubble

In [4]:
# transform data
log_prices = np.array(np.log(prices))
delta_log_prices = log_prices[1:] - log_prices[:-1] # log returns
n = len(delta_log_prices)

In [5]:
# run BSADF : Backaward Subsample Augmented Dickey Fuller test
# DF test over backward rolling subsamples of prices, in order to timestamp the bubble
# we look at the right tail of the Dickey Fuller T-distribution : if its above the
# critical value, then the return generating process is explosive

BSADF = np.array([])
for r2 in range(r0,n): # for each size of subsamples...
  ADFS = np.array([])  # ...we compute ADF stat
  for r1 in range(0,r2-r0+1):
    X0 = log_prices[r1:r2+1]
    X = pd.DataFrame()
    X[0] = X0
    for j in range(1,adf_lags+1):
      X[j] = np.append(np.zeros(j), delta_log_prices[r1:r2+1-j])
    X = np.array(X)
    Y = delta_log_prices[r1:r2+1]
    reg = sm.OLS(Y, sm.add_constant(X))
    res = reg.fit()
    ADFS = np.append(ADFS, res.params[1]/res.bse[1]) # tsat = param/standard_error
  BSADF = np.append(BSADF, max(ADFS)) # max ADF Stat for the subsample

In [15]:
# printing dates when a bubble is detected
print(prices.index[r0+1:][BSADF > crit])

DatetimeIndex(['2020-11-21', '2020-11-23', '2020-11-24', '2021-01-02',
               '2021-01-03', '2021-01-04', '2021-01-05', '2021-01-06',
               '2021-01-28', '2021-01-29', '2021-01-30', '2021-01-31',
               '2021-02-01', '2021-02-02', '2021-02-03', '2021-02-04',
               '2021-02-05', '2021-02-06', '2021-02-07', '2021-02-08',
               '2021-02-09', '2021-02-10', '2021-02-11', '2021-02-12',
               '2021-02-13', '2021-02-14', '2021-02-15', '2021-02-16',
               '2021-02-18', '2021-04-15', '2021-04-16', '2021-04-17',
               '2021-04-18', '2021-04-19', '2021-04-20', '2021-04-21',
               '2021-04-24', '2021-04-26', '2021-04-28', '2021-04-29',
               '2021-04-30', '2021-05-01', '2021-05-02', '2021-05-03',
               '2021-05-04', '2021-05-05', '2021-05-06', '2021-05-07',
               '2021-05-08', '2021-05-09', '2021-05-10', '2021-05-11',
               '2021-05-13', '2021-05-14', '2021-05-15', '2021-05-16',
      

In [14]:
# visualising results
import plotly.graph_objects as go

# Create figure
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=list(prices.index[r0+1:]), y=list(prices[r0+1:]), name = "Price"))
fig.add_trace(
    go.Scatter(x=list(prices.index[r0+1:]), y=list(BSADF), name = "BSADF"))
fig.add_trace(
    go.Scatter(x=list(prices.index[r0+1:]), y=list(np.ones(len(BSADF))*crit), name = "BSADF - critical value"))

# Set title
fig.update_layout(
    title_text="Detecting Bubbles for " + ticker + " between " + start + " and " + end
)

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()